In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### 주어진 문장을 '단어'로 토큰화하기

In [ ]:
# 텍스트 전처리 함수 text_to_word_sequence(), Tokenizer() 호출

from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

In [ ]:
# 전처리할 텍스트 정하기
text = '해 보지 않으면 해낼 수 없다'

# 해당 텍스트 토큰화
result = text_to_word_sequence(text)
print('\n원문:\n',text)
print('\n토큰화:\n',result)


원문:
 해 보지 않으면 해낼 수 없다

토큰화:
 ['해', '보지', '않으면', '해낼', '수', '없다']


In [ ]:
# 전처리하려는 세 개의 문장 정하기

docs={'먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
      '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
      '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.'}

In [ ]:
# 토큰화 함수를 이용해 전처리하는 과정
token = Tokenizer()
token.fit_on_texts(docs)

# 각 옵션에 맞춰 단어의 빈도 수를 계산한 결과 출력
print('\n단어 카운트:\n', token.word_counts)


단어 카운트:
 OrderedDict([('토큰화한', 1), ('결과는', 1), ('딥러닝에서', 2), ('사용할', 1), ('수', 1), ('있습니다', 1), ('텍스트의', 2), ('단어로', 1), ('토큰화해야', 1), ('인식됩니다', 1), ('먼저', 1), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1)])


In [ ]:
# 출력되는 순서는 랜덤
print('\n문장 카운트:\n', token.document_count)
print('\n각 단어가 몇 개의 문장에 포함되어 있는가:\n', token.word_docs)
print('\n각 단어에 매겨진 인덱스 값:\n', token.word_index)


문장 카운트:
 3

각 단어가 몇 개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'딥러닝에서': 2, '사용할': 1, '토큰화한': 1, '수': 1, '결과는': 1, '있습니다': 1, '텍스트의': 2, '단어로': 1, '인식됩니다': 1, '토큰화해야': 1, '먼저': 1, '토큰화합니다': 1, '각': 1, '단어를': 1, '나누어': 1})

각 단어에 매겨진 인덱스 값:
 {'딥러닝에서': 1, '텍스트의': 2, '토큰화한': 3, '결과는': 4, '사용할': 5, '수': 6, '있습니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '먼저': 11, '각': 12, '단어를': 13, '나누어': 14, '토큰화합니다': 15}


#### 영화 리뷰가 긍정적인지 부정적인지 예측하기

In [ ]:
import numpy
import tensorflow as tf

from numpy import array                                           # 배열화
from tensorflow.keras.preprocessing.text import Tokenizer         # 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences # 단어 임베딩
from tensorflow.keras.models import Sequential                    # 모델 설정
from tensorflow.keras.layers import Dense,Flatten,Embedding

In [ ]:
# 텍스트 리뷰 자료 지정
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다",
        "한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

In [ ]:
# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스 지정
classes = array([1,1,1,1,1,0,0,0,0,0])

In [ ]:
# 단어 토큰화 
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [ ]:
x = token.texts_to_sequences(docs)
print(x) 

[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [ ]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞추어 줍니다.
padded_x = pad_sequences(x, 4)  
print("\n패딩 결과:\n", padded_x)


패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [ ]:
#임베딩에 입력될 단어 수 지정
word_size = len(token.word_index) +1

In [ ]:
 #단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과 출력
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.fit(padded_x, classes, epochs=20)

Epoch 1/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6879 - accuracy: 0.6000
Epoch 2/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6858 - accuracy: 0.6000
Epoch 3/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6838 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 1ms/step - loss: 0.6817 - accuracy: 0.7000
Epoch 5/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6796 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6776 - accuracy: 0.8000
Epoch 7/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6755 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 1ms/step - loss: 0.6735 - accuracy: 0.9000
Epoch 9/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6714 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6694 - accuracy: 0.9000
Epoch 11/20
1/1 [======================

In [ ]:
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

1/1 [==============================] - 0s 2ms/step - loss: 0.6462 - accuracy: 0.9000

 Accuracy: 0.9000
